In [2]:
import ta
import ccxt
import time
import pprint
import pandas as pd
import SecretConfig
from IPython.display import clear_output

In [3]:
# Définir l'échange
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [8]:
futures_tickers = exchange.fetc(params={'type': 'swap'})
pprint.pprint(futures_tickers)

{'10000LADYS/USDT:USDT': {'ask': 0.0003525,
                          'askVolume': 237100.0,
                          'average': 0.00035395,
                          'baseVolume': 1300190100.0,
                          'bid': 0.0003521,
                          'bidVolume': 237200.0,
                          'change': -3.1e-06,
                          'close': 0.0003524,
                          'datetime': '2023-11-12T10:35:29.854Z',
                          'high': 0.0003789,
                          'info': {'askSz': '237100',
                                   'baseVolume': '1300190100',
                                   'bestAsk': '0.0003525',
                                   'bestBid': '0.0003521',
                                   'bidSz': '237200',
                                   'chgUtc': '-0.00872',
                                   'deliveryStatus': 'normal',
                                   'fundingRate': '0.0001',
                                   'hig

In [24]:
# Initialiser un DataFrame vide pour stocker les données roulantes
full_rolling_dataframe = pd.DataFrame()

# Définir une fonction pour récupérer et traiter les données
def fetch_data():
    global full_rolling_dataframe
    while True:
        # Récupérer les données des contrats à terme
        futures_tickers = exchange.fetch_tickers(params={'type': 'swap'})
        tickers_name_list = futures_tickers.keys()

        # Initialiser un dictionnaire pour stocker les nouvelles données
        data = {}

        # Parcourir chaque contrat à terme
        for ticker in tickers_name_list:
            # Récupérer les informations pertinentes pour chaque contrat à terme
            ticker_name = ticker.split('/')[0]
            data[ticker_name + ' Last Price'] = futures_tickers.get(ticker, {}).get('info', {}).get('last')
            data[ticker_name + ' Quote Volume'] = futures_tickers.get(ticker, {}).get('quoteVolume')

            # Initialiser les indicateurs techniques et les variations à None
            indicators = [' High', ' Low', ' Close', ' Volume Ticker', ' SMA_1', ' SMA_2', ' EMA_1', ' EMA_2', ' RSI_1', ' RSI_2', ' MACD', ' ATR', ' Price Change', ' Volume Change', ' ATR Change']
            for indicator in indicators:
                data[ticker_name + indicator] = None

        # Convertir le dictionnaire en DataFrame
        data['Timestamp'] = futures_tickers.get(ticker, {}).get('timestamp')
        new_data_df = pd.DataFrame([data])

        # Convertir les colonnes en numérique et définir 'Timestamp' comme index
        new_data_df = new_data_df.apply(pd.to_numeric)
        new_data_df['Timestamp'] = pd.to_datetime(new_data_df['Timestamp'], unit='ms')
        new_data_df['Timestamp'] = new_data_df['Timestamp'].dt.round('10L')
        new_data_df = new_data_df.set_index('Timestamp')

        # Ajouter les nouvelles données au DataFrame roulant
        full_rolling_dataframe = pd.concat([full_rolling_dataframe, new_data_df])

        # Calculer les indicateurs techniques
        for ticker in tickers_name_list:
            ticker_name = ticker.split('/')[0]
            full_rolling_dataframe[ticker_name + ' High'] = full_rolling_dataframe[ticker_name + ' Last Price'].rolling(window=10).max()
            full_rolling_dataframe[ticker_name + ' Low'] = full_rolling_dataframe[ticker_name + ' Last Price'].rolling(window=10).min()
            full_rolling_dataframe[ticker_name + ' Close'] = full_rolling_dataframe[ticker_name + ' Quote Volume'].rolling(window=10).apply(lambda x: x.iloc[-1])
            full_rolling_dataframe[ticker_name + ' Volume Ticker'] = full_rolling_dataframe[ticker_name + ' Quote Volume'].diff()
            full_rolling_dataframe[ticker_name + ' SMA_1'] = ta.trend.sma_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window=2)
            full_rolling_dataframe[ticker_name + ' SMA_2'] = ta.trend.sma_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window=4)
            full_rolling_dataframe[ticker_name + ' EMA_1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window=2)
            full_rolling_dataframe[ticker_name + ' EMA_2'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window=4)
            full_rolling_dataframe[ticker_name + ' RSI_1'] = ta.momentum.rsi(full_rolling_dataframe[ticker_name + ' Last Price'])
            full_rolling_dataframe[ticker_name + ' RSI_2'] = ta.momentum.rsi(full_rolling_dataframe[ticker_name + ' Last Price']) 
            full_rolling_dataframe[ticker_name + ' MACD'] = ta.trend.macd(full_rolling_dataframe[ticker_name + ' Last Price'])
            if len(full_rolling_dataframe) >= 14:
                full_rolling_dataframe[ticker_name + ' ATR'] = ta.volatility.average_true_range(full_rolling_dataframe[ticker_name + ' High'], full_rolling_dataframe[ticker_name + ' Low'], full_rolling_dataframe[ticker_name + ' Close'])
            else:
                full_rolling_dataframe[ticker_name + ' ATR'] = None
            full_rolling_dataframe[ticker_name + ' Price Change'] = full_rolling_dataframe[ticker_name + ' Last Price'].pct_change(fill_method=None)
            full_rolling_dataframe[ticker_name + ' Volume Change'] = full_rolling_dataframe[ticker_name + ' Volume Ticker'].pct_change(fill_method=None)
            full_rolling_dataframe[ticker_name + ' ATR Change'] = full_rolling_dataframe[ticker_name + ' ATR'].rolling(window=10).apply(lambda x: x.iloc[-1] / x.iloc[0] - 1 if len(x) == 10 else None)
            full_rolling_dataframe = full_rolling_dataframe.copy()

        # Limiter le DataFrame roulant aux 600 dernières périodes
        full_rolling_dataframe = full_rolling_dataframe.tail(600)

        # Afficher les 10 dernières lignes du DataFrame
        clear_output(wait=True)
        display(full_rolling_dataframe.tail(20))
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR'])
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR Change'])
        # pprint.pprint(full_rolling_dataframe)

        # Attendre 1 seconde avant de récupérer à nouveau les données
        time.sleep(0)

fetch_data()

,BTC Last Price,BTC Quote Volume,BTC High,BTC Low,BTC Close,BTC Volume Ticker,BTC SMA_1,BTC SMA_2,BTC EMA_1,BTC EMA_2,...,MEME SMA_2,MEME EMA_1,MEME EMA_2,MEME RSI_1,MEME RSI_2,MEME MACD,MEME ATR,MEME Price Change,MEME Volume Change,MEME ATR Change
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-12 00:28:46.310,37032.6,1.900845e+09,37033.3,37012.4,1.900845e+09,47993.5913,37032.20,37031.300,37032.102573,37029.558282,...,0.021478,0.021475,0.021474,29.418805,29.418805,NaN,2.848729e+07,0.000000,-1.000000,inf
2023-11-12 00:28:49.510,37036.4,1.901190e+09,37036.4,37012.4,1.901190e+09,344119.8896,37034.50,37033.525,37034.967524,37032.294969,...,0.021476,0.021474,0.021474,29.418805,29.418805,NaN,2.881356e+07,0.000000,NaN,inf
2023-11-12 00:28:52.850,37036.4,1.901466e+09,37036.4,37012.4,1.901466e+09,276921.1628,37036.40,37034.300,37035.922508,37033.936981,...,0.021474,0.021474,0.021474,29.418805,29.418805,NaN,2.911652e+07,0.000000,NaN,inf
2023-11-12 00:28:56.340,37033.8,1.901525e+09,37036.4,37012.4,1.901525e+09,58961.7469,37035.10,37034.800,37034.507503,37033.882189,...,0.021479,0.021487,0.021482,48.372830,48.372830,NaN,2.939784e+07,0.000931,inf,inf
2023-11-12 00:28:59.600,37034.5,1.901827e+09,37036.4,37012.4,1.901827e+09,301427.9868,37034.15,37035.275,37034.502501,37034.129313,...,0.021491,0.021510,0.021498,62.869342,62.869342,NaN,2.965907e+07,0.001256,16.939795,0.121787
2023-11-12 00:29:02.870,37035.4,1.900875e+09,37036.4,37012.8,1.900875e+09,-951366.9398,37034.95,37035.025,37035.100834,37034.637588,...,0.021503,0.021517,0.021507,62.869342,62.869342,NaN,2.990170e+07,0.000000,-24.468701,0.111119
2023-11-12 00:29:06.370,37033.9,1.900877e+09,37036.4,37027.5,1.900877e+09,1925.7772,37034.65,37034.400,37034.300278,37034.342553,...,0.021514,0.021520,0.021513,62.869342,62.869342,NaN,3.012575e+07,0.000000,-1.000000,0.101488
2023-11-12 00:29:09.860,37033.8,1.900884e+09,37036.4,37031.8,1.900884e+09,6592.0342,37033.85,37034.400,37033.966759,37034.125532,...,0.021521,0.021521,0.021516,62.869342,62.869342,-1.514419e-06,3.033380e+07,0.000000,NaN,0.092813
2023-11-12 00:29:13.280,37033.9,1.900950e+09,37036.4,37031.8,1.900950e+09,66438.6593,37033.85,37034.250,37033.922253,37034.035319,...,0.021521,0.021520,0.021518,62.002048,62.002048,1.456187e-07,3.052700e+07,-0.000046,inf,0.084983


KeyboardInterrupt: 